In [1]:
import pandas as pd
import numpy as np
ruta_input = "../file/output/"
df_Estudiante = pd.read_csv(ruta_input+'SQL_tabla_1.csv',index_col=False, skiprows=2)
df_Ciudad = pd.read_csv(ruta_input+'SQL_tabla_2.csv', skiprows=2)
ruta_output_clean = "../data/"

In [2]:
df_Estudiante.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              1000 non-null   int64 
 1   NOMBREAPELLIDO  1000 non-null   object
 2   CODIGOCIUDAD    1000 non-null   int64 
 3   FECHAINGRESO    995 non-null    object
 4   TELEFONO        992 non-null    object
 5   CORREO          1000 non-null   object
 6   CARRERA         1000 non-null   object
dtypes: int64(2), object(5)
memory usage: 54.8+ KB


In [3]:
if df_Estudiante['ID'].is_unique:
    print("El ID de estudiante es único.")

carreras = df_Estudiante['CARRERA'].unique()
print("Carreras únicas:", carreras)


El ID de estudiante es único.
Carreras únicas: ['Ingeniería de sistemas' 'Ingenieria química' 'Matemáticas'
 'Ciencia política' 'Estadística' 'Geología ' 'Ingeniería Industrial'
 'Administración de empresas' 'Ingeniería eléctrica'
 'Ingenieria electrónica']


In [4]:
dominios_default = df_Estudiante['CORREO'].str.split('@').str[1]
print("Dominios por defecto:", dominios_default.unique())

casos_sin_arroba = df_Estudiante[~df_Estudiante['CORREO'].str.contains('@', na=False)][['ID', 'CORREO']]
print(casos_sin_arroba)

# Remplazara correo de "ID == 6" por NaN dado que no tiene arroba, y podria confundir que el _ es parte para la @
df_Estudiante.loc[df_Estudiante['ID'] == 6, 'CORREO'] = np.nan


Dominios por defecto: ['hotmail.com' 'hotmail.c' nan 'gmail.com' 'outlook.com' 'outlook.co'
 'hotmail.co' 'gmail.co' 'gmail.' 'gmail' '.co']
   ID                 CORREO
5   6  odyuliana_hotmail.com


In [5]:
# Estos dominios hotmail, gmail, outlook son validos sin terminan en .com 
df_Estudiante['CORREO'] = df_Estudiante['CORREO'].str.replace(r'@.*(hotmail|gmail|outlook).*', r'@\1.com', regex=True).where(df_Estudiante['CORREO'].str.contains(r'@(hotmail|gmail|outlook)', na=False))

C:\Users\BRAYAN\AppData\Local\Temp\ipykernel_14104\4219987166.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_Estudiante['CORREO'] = df_Estudiante['CORREO'].str.replace(r'@.*(hotmail|gmail|outlook).*', r'@\1.com', regex=True).where(df_Estudiante['CORREO'].str.contains(r'@(hotmail|gmail|outlook)', na=False))


In [6]:
correos_a_reemplazar = df_Estudiante[df_Estudiante['CORREO'].isin(['@gmail.com', '@hotmail.com', '@outlook.com'])][['ID', 'CORREO']]
print("Correos a reemplazar:\n", correos_a_reemplazar)
df_Estudiante.loc[df_Estudiante['ID'].isin(correos_a_reemplazar['ID']), 'CORREO'] = np.nan

Correos a reemplazar:
       ID        CORREO
7      8  @hotmail.com
228  229    @gmail.com


In [7]:
df = df_Estudiante['TELEFONO'].astype(str).str.replace(r'\D', '', regex=True).str.len()

conteo_digitos = df.value_counts().sort_index()
print("Cantidad de dígitos y frecuencia:")
for digitos, cantidad in conteo_digitos.items():
    print(f"{digitos} dígitos: {cantidad} teléfonos")

print(f"\nMás común: {conteo_digitos.index[conteo_digitos.argmax()]} dígitos")


Cantidad de dígitos y frecuencia:
0 dígitos: 8 teléfonos
4 dígitos: 2 teléfonos
5 dígitos: 2 teléfonos
6 dígitos: 1 teléfonos
7 dígitos: 987 teléfonos

Más común: 7 dígitos


In [8]:
# formatear la columana df_Estudiante['TELEFONO'] a entero si tiene 7 digitos, sino convertirlo a NaN
df_Estudiante['TELEFONO'] = df_Estudiante['TELEFONO'].astype(str).str.replace(r'\D', '', regex=True)
df_Estudiante['TELEFONO'] = df_Estudiante['TELEFONO'].apply(lambda x: int(x) if len(x) == 7 else pd.NA)
df_Estudiante.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID              1000 non-null   int64 
 1   NOMBREAPELLIDO  1000 non-null   object
 2   CODIGOCIUDAD    1000 non-null   int64 
 3   FECHAINGRESO    995 non-null    object
 4   TELEFONO        987 non-null    object
 5   CORREO          996 non-null    object
 6   CARRERA         1000 non-null   object
dtypes: int64(2), object(5)
memory usage: 54.8+ KB


In [9]:
# FECHAINGRESO: object → debería ser datetime sin tener en cuenta la hora
df_Estudiante['FECHAINGRESO'] = pd.to_datetime(df_Estudiante['FECHAINGRESO'], errors='coerce').dt.date
condiciones = [
    df_Estudiante['FECHAINGRESO'].isnull(),
    df_Estudiante['TELEFONO'].isnull(),
    df_Estudiante['CORREO'].isnull()
]

opciones = ['FECHAINGRESO', 'TELEFONO', 'CORREO']
df_Estudiante['CONTACTO_INCOMPLETO'] = np.select(condiciones, opciones, default=None)


In [10]:
mapa_ciudades = df_Ciudad.set_index('CODIGOCIUDAD')['NOMBRECIUDAD']
df_Estudiante['NOMBRECIUDAD'] = df_Estudiante['CODIGOCIUDAD'].map(mapa_ciudades)

columna_ciudad = df_Estudiante.pop('NOMBRECIUDAD')
posicion_insercion = df_Estudiante.columns.get_loc('CONTACTO_INCOMPLETO')
df_Estudiante.insert(loc=posicion_insercion, column='NOMBRECIUDAD', value=columna_ciudad)

conteo_estudiantes = df_Estudiante['CODIGOCIUDAD'].value_counts()
df_Ciudad['ESTUDIANTES'] = df_Ciudad['CODIGOCIUDAD'].map(conteo_estudiantes).fillna(0).astype(int)
#Ordenar por "Estudiantes"
df_Ciudad = df_Ciudad.sort_values(by='ESTUDIANTES', ascending=False)


In [11]:
df_Estudiante.to_csv(ruta_output_clean+'Estudiante.csv', index=False)
df_Ciudad.to_csv(ruta_output_clean+'Ciudad.csv', index=False)